In [25]:
import matplotlib as mpl

# This line allows mpl to run with no DISPLAY defined
mpl.use('Agg')

import pandas as pd
import numpy as np
import os
from keras.layers import Reshape, Flatten, LeakyReLU, Activation
from keras.layers.convolutional import UpSampling2D, MaxPooling2D
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from keras_adversarial.image_grid_callback import ImageGridCallback

from keras_adversarial import AdversarialModel, simple_gan, gan_targets
from keras_adversarial import AdversarialOptimizerSimultaneous, normal_latent_sampling
from keras_adversarial.legacy import Dense, BatchNormalization, fit, l1l2, Convolution2D, AveragePooling2D
from keras.layers import Conv2D
import keras.backend as K
from keras.datasets import cifar10
#from cifar10_utils import cifar10_data
#from image_utils import dim_ordering_fix, dim_ordering_unfix, dim_ordering_shape

import numpy as np
import matplotlib.pyplot as plt
import scipy.misc
import os
import imageio

In [30]:
class GAN(object):
    def __init__(self, width = 32, height= 32, channels = 3):

        self.WIDTH = width
        self.HEIGHT = height
        self.CHANNELS = channels

        self.SHAPE = (self.WIDTH, self.HEIGHT, self.CHANNELS)

        self.OPTIMIZER = Adam(lr=0.0002, decay=8e-9)

        self.noise_gen = np.random.normal(0,1,(100,))

        self.G = self.generator()
        self.G.compile(loss='binary_crossentropy', optimizer=self.OPTIMIZER)

        self.D = self.discriminator()
        self.D.compile(loss='binary_crossentropy', optimizer=self.OPTIMIZER, metrics=['accuracy'] )

        self.stacked_G_D = self.stacked_G_D()

        self.stacked_G_D.compile(loss='binary_crossentropy', optimizer=self.OPTIMIZER)

    # returns generator (produces new images)
    def generator(self):
        model = Sequential()
        nch = 256
        reg = lambda: l1l2(l1=1e-7, l2=1e-7)
        h = 5
        model.add(Dense(nch * 4 * 4, input_dim=100))
        model.add(BatchNormalization(mode=0))
        model.add(Reshape((4, 4, nch)))
        model.add(Conv2D(int(nch / 2), (h, h), border_mode='same'))
        model.add(BatchNormalization(mode=0, axis=1))
        model.add(LeakyReLU(0.2))
        model.add(UpSampling2D(size=(2, 2)))
        model.add(Conv2D(int(nch / 2), (h, h), border_mode='same'))
        model.add(BatchNormalization(mode=0, axis=1))
        model.add(LeakyReLU(0.2))
        model.add(UpSampling2D(size=(2, 2)))
        model.add(Conv2D(int(nch / 4), (h, h), border_mode='same'))
        model.add(BatchNormalization(mode=0, axis=1))
        model.add(LeakyReLU(0.2))
        model.add(UpSampling2D(size=(2, 2)))
        model.add(Conv2D(3, (h, h), border_mode='same'))
        model.add(Activation('sigmoid'))
        model.summary()
        return model
    
    
    # returns discriminator (distincts real images from generated)
    def discriminator(self):
        nch = 256
        h = 5
        reg = lambda: l1l2(l1=1e-7, l2=1e-7)

        c1 = Conv2D(h, h, int(nch / 4), border_mode='same', W_regularizer=reg(),
                           input_shape=self.SHAPE)
        c2 = Conv2D(int(nch / 2), (h, h), border_mode='same', W_regularizer=reg())
        c3 = Conv2D(nch, (h, h), border_mode='same', W_regularizer=reg())
        c4 = Conv2D(1, (h, h), border_mode='same', W_regularizer=reg())

        model = Sequential()
        model.add(c1)
        model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th"))
        model.add(LeakyReLU(0.2))
        model.add(c2)
        model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th"))
        model.add(LeakyReLU(0.2))
        model.add(c3)
        model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th"))
        model.add(LeakyReLU(0.2))
        model.add(c4)
        model.add(AveragePooling2D(pool_size=(2, 2), border_mode='valid'))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        return model

    
    # stacks generator and discriminator (discriminator is not trainable)
    def stacked_G_D(self):
        self.D.trainable = False # discriminator is trained separately

        model = Sequential()
        model.add(self.G)
        model.add(self.D)

        return model

    
    
    def train(self, X_train, epochs=20000, batch = 512, save_interval = 100):

        d_loss = 500
        g_loss = 500
        for cnt in range(30000):

            if cnt % 100 == 0:
                for ix in range(10):
                    ## train discriminator
                    random_index =  np.random.randint(0, len(X_train) - int(batch/2))
                    legit_images = X_train[random_index : random_index + int(batch/2)]
                    legit_images = legit_images.reshape(int(batch/2), self.WIDTH, self.HEIGHT, self.CHANNELS)

                    gen_noise = np.random.normal(0, 1, (int(batch/2),100))
                    syntetic_images = self.G.predict(gen_noise)

                    x_combined_batch = np.concatenate((legit_images, syntetic_images))
                    y_combined_batch = np.concatenate((np.ones((int(batch/2), 1)), np.zeros((int(batch/2), 1))))

                    d_loss = self.D.train_on_batch(x_combined_batch, y_combined_batch)
                    print("Generator {}/10".format(ix))
                print()

                    
            else:
                # train generator
                noise = np.random.normal(0, 1, (batch,100))
                y_mislabled = np.ones((batch, 1))

                g_loss = self.stacked_G_D.train_on_batch(noise, y_mislabled)
                print('.', end='')

            if cnt % 1 == 0:
                print ('epoch: %d, [Discriminator :: d_loss: %f] [ Generator :: loss: %f]' % (cnt, d_loss[0], g_loss))

            if cnt % 10 == 0 : 
                self.save_images(cnt=cnt)

    def save_images(self, cnt, samples=10):
        
        noise = np.random.normal(0, 1, (samples,100))
        images = self.G.predict(noise)
        
        for ix, image in enumerate(images):
            if not os.path.exists('results/{:05d}'.format(cnt)):
                os.makedirs('results/{:05d}'.format(cnt))
            imageio.imwrite('results/{:05d}/{}.jpg'.format(cnt, ix), image)

In [7]:
(X_train, y_train), (_, _) = cifar10.load_data()
X_train = X_train[np.where(y_train == 0)[0]]
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
#X_train = np.expand_dims(X_train, axis=3)

# X_train = np.zeros((50000, 32, 32, 3))
# X_train[:, 10:22, 10:22] = 1

In [31]:
X_train = np.ones((5000, 32, 32, 3))
X_train[:, 10:22, 10:22, :] = 0.

In [32]:
X_train.shape

(5000, 32, 32, 3)

In [33]:
gan = GAN()
gan.train(X_train)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5), padding="same")`
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(3, (5, 5), padding="same")`
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(5, (5, 64), input_shape=(32, 32, 3..., padding="same", kernel_regularizer=<keras.reg...)`
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:59: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5), padding="same", kernel_regularizer=<keras.reg...)`
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:60: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (5, 5), padding="same", kernel_regularizer=<keras.reg...)`
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:61:

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 4096)              413696    
_________________________________________________________________
batch_normalization_13 (Batc (None, 4096)              16384     
_________________________________________________________________
reshape_4 (Reshape)          (None, 4, 4, 256)         0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 4, 4, 128)         819328    
_________________________________________________________________
batch_normalization_14 (Batc (None, 4, 4, 128)         16        
_________________________________________________________________
leaky_re_lu_19 (LeakyReLU)   (None, 4, 4, 128)         0         
_________________________________________________________________
up_sampling2d_10 (UpSampling (None, 8, 8, 128)         0         
__________

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:68: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:71: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`


ValueError: Negative dimension size caused by subtracting 2 from 1 for 'sequential_11/average_pooling2d_4/AvgPool' (op: 'AvgPool') with input shapes: [?,8,1,1].